# Voca prompt

In [21]:
from pydantic import BaseModel, create_model
from openai import OpenAI
from openai.lib._parsing import type_to_response_format_param
import json
from getpass import getpass
from jinja2 import Template
import re
from enum import Enum
from typing import Optional

In [22]:
openai_api_key = getpass('OPENAI_API_KEY')
client = OpenAI(api_key=openai_api_key)

In [23]:
prompt_template = Template("""
[GUIDELINE]

Provide matching "VOCA_OPTIONS".
If the "WORD" of the input has more than one match in the lists of "VOCA_OPTIONS", judge with ["PART_OF_SPEECH", "POS_NO", "MEANING", "EXAMPLE_SENTENCE", "EXAMPLE_SENTENCE_KOR"] to indicate the most adequate match.
If the "WORD" of the input has NO match from "VOCA_OPTIONS", leave the voca_id as null.

                           
[EXAMPLE]
```
Input:
    {
    "TEXTBOOK_ID" : "701",
    "LESSON" : "3",
    "WORD" : "donate",
    "PART_OF_SPEECH" : "동사",
    "POS_NO" : "1"                    
    "MEANING" : "기부하다, 기증하다",
    "EXAMPLE_SENTENCE" : "I donate money to many different charities each year.",
    "EXAMPLE_SENTENCE_KOR" : "나는 매년 여러 다양한 자선 단체에 돈을 기부한다.",
    "VOCA_OPTIONS" : "[[1093, "donate", "헌혈하다,(장기를) 기증하다", 1, "동사"], [1094, "donate", "기부하다", 1, "동사"]]" 
    }

Output:
    {
        "voca_id": 1094
    }
'''
```
Input:
    {
    "TEXTBOOK_ID" :"802",
    "LESSON" : "3",
    "WORD" : "get",
    "PART_OF_SPEECH" : "동사",
    "POS_NO" : "1"                    
    "MEANING" : "도착하다",
    "EXAMPLE_SENTENCE" : "He got to Europe to perform.",
    "EXAMPLE_SENTENCE_KOR" : "그는 공연을 위해 유럽에 도착했어요.",
    "VOCA_OPTIONS" : "[[2914, "get", "얻다", 1, "동사"], [2915, "get", "이해하다", 1, "동사"]]" 
    }

Output:
    {
        "voca_id": null
    }
'''                           
                           
[Input]
```
TEXTBOOK_ID : {{textbook_id}}
LESSON : {{lesson}}
WORD : {{word}}
PART_OF_SPEECH : {{part_of_speech}}
POS_NO : {{pos_no}}
MEANING: {{meaning}}
EXAMPLE_SENTENCE: {{example_sentence}}
EXAMPLE_SENTENCE_KOR: {{example_sentence_kor}}
VOCA_OPTIONS: {{voca_options}}
```                   
"""
)

In [24]:
voca_option_list = eval('[[2624, "discount", "할인", 2, "명사"], [2625, "discount", "할인하다", 1, "동사"]]')

In [25]:
voca_option_list

[[2624, 'discount', '할인', 2, '명사'], [2625, 'discount', '할인하다', 1, '동사']]

In [26]:
voca_id_list = [print(x[0]) for x in voca_option_list]

2624
2625


In [27]:
prompt = prompt_template.render(
    textbook_id = "807",
    lesson = "5",
    word = "charge",
    part_of_speech = "명사",
    pos_no = "2",
    meaning = "담당, 책임",
    example_sentence = "He took charge of the farm after his father's death.",
    example_sentence_kor = "그는 아버지가 죽고 나서 농장을 책임지게 되었다.",
    voca_options = '[[1843, "charge", "청구하다", 1, "동사"], [1844, "charge", "충전하다", 1, "동사"], [1845, "charge", "책임을 맡기다", 1, "동사"], [1846, "charge", "기소하다, 고소하다", 1, "동사"], [1847, "charge", "(외상으로) 달아놓다", 1, "동사"], [1848, "charge", "비난", 2, "명사"], [1849, "charge", "기소, 혐의", 2, "명사"], [1850, "charge", "요금", 2, "명사"]]'
)

In [28]:
print(prompt)


[GUIDELINE]

Provide matching "VOCA_OPTIONS".
If the "WORD" of the input has more than one match in the lists of "VOCA_OPTIONS", judge with ["PART_OF_SPEECH", "POS_NO", "MEANING", "EXAMPLE_SENTENCE", "EXAMPLE_SENTENCE_KOR"] to indicate the most adequate match.
If the "WORD" of the input has NO match from "VOCA_OPTIONS", leave the voca_id as null.

                           
[EXAMPLE]
```
Input:
    {
    "TEXTBOOK_ID" : "701",
    "LESSON" : "3",
    "WORD" : "donate",
    "PART_OF_SPEECH" : "동사",
    "POS_NO" : "1"                    
    "MEANING" : "기부하다, 기증하다",
    "EXAMPLE_SENTENCE" : "I donate money to many different charities each year.",
    "EXAMPLE_SENTENCE_KOR" : "나는 매년 여러 다양한 자선 단체에 돈을 기부한다.",
    "VOCA_OPTIONS" : "[[1093, "donate", "헌혈하다,(장기를) 기증하다", 1, "동사"], [1094, "donate", "기부하다", 1, "동사"]]" 
    }

Output:
    {
        "voca_id": 1094
    }
'''
```
Input:
    {
    "TEXTBOOK_ID" :"802",
    "LESSON" : "3",
    "WORD" : "get",
    "PART_OF_SPEECH" : "동사",
    "POS_N

In [29]:
from enum import Enum
from pydantic import create_model
from openai.lib._parsing import type_to_response_format_param
from typing import Optional

def get_enum_response_format(voca_id_list: list[int]):
    enum_dict = {str(x): x for x in voca_id_list}
    VocaIdEnum = Enum("VocaIdEnum", enum_dict) # 동적 enum 클래스 생성
    response_format_dict = {
        "voca_id": (Optional[VocaIdEnum], ...)
    }
    ResponseModel = create_model("VocaIdResponse", **response_format_dict)
    return type_to_response_format_param(ResponseModel)

- response format임 -> 출력만 쉽게 나온다는 것
- prompt에 잘 작성을 해야할 것

In [30]:
voca_id_list

[None, None]

In [31]:
enum_response_format = get_enum_response_format(voca_id_list)

In [32]:
enum_response_format

{'type': 'json_schema',
 'json_schema': {'schema': {'$defs': {'VocaIdEnum': {'const': None,
     'enum': [None],
     'title': 'VocaIdEnum'}},
   'properties': {'voca_id': {'anyOf': [{'$ref': '#/$defs/VocaIdEnum'},
      {'type': 'null'}]}},
   'required': ['voca_id'],
   'title': 'VocaIdResponse',
   'type': 'object',
   'additionalProperties': False},
  'name': 'VocaIdResponse',
  'strict': True}}

In [33]:
import json

def completion(prompt : str, response_format: str) -> str:
    response = client.beta.chat.completions.parse(
        model = 'o3-mini-2025-01-31',
        reasoning_effort='low',
        messages = [
            {"role" : "system", "content" : "Choose the most adequate word"},
            {"role" : "user", "content" : prompt}
        ],
        response_format=response_format,
    )
    generated_str = response.choices[0].message.content
    generated_json = json.loads(generated_str)
    return generated_json

In [34]:
response = completion(prompt, enum_response_format)

BadRequestError: Error code: 400 - {'error': {'message': "Invalid schema: Objects provided via 'anyOf' must not share identical first keys. Consider adding a discriminator key or rearranging the properties to ensure the first key is unique.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [ ]:
print(response)

{'voca_id': None}


In [ ]:
# response_output = json.dumps(response.dict(), ensure_ascii=False, indent=2)
# print(response_output)

# Sample Data 만들기

In [ ]:
import pandas as pd
data = [
    [808, 8, "charge", "동사", 1, "(지불, 대금 등을) 청구하다", "They charge you $20 to get in the museum.", "그들은 박물관에 들어가는 데 너에게 20달러를 청구한다.",[[1843, "charge", "청구하다", 1, "동사"], [1844, "charge", "충전하다", 1, "동사"], [1845, "charge", "책임을 맡기다", 1, "동사"], [1846, "charge", "기소하다, 고소하다", 1, "동사"], [1847, "charge", "(외상으로) 달아놓다", 1, "동사"], [1848, "charge", "비난", 2, "명사"], [1849, "charge", "기소, 혐의", 2, "명사"], [1850, "charge", "요금", 2, "명사"]]],
    [903, 6, "charge", "동사", 1, "(요금을) 청구하다", "They charge you $5 to get in the museum.", "그들은 박물관에 들어가는 데 5달러를 당신에게 부과할 것이다.",[[1843, "charge", "청구하다", 1, "동사"], [1844, "charge", "충전하다", 1, "동사"], [1845, "charge", "책임을 맡기다", 1, "동사"], [1846, "charge", "기소하다, 고소하다", 1, "동사"], [1847, "charge", "(외상으로) 달아놓다", 1, "동사"], [1848, "charge", "비난", 2, "명사"], [1849, "charge", "기소, 혐의", 2, "명사"], [1850, "charge", "요금", 2, "명사"]]],
    [906, 5, "charge", "명사", 2, "기소, 혐의", "He was arrested on the charge of robbery.", "그는 강도 혐의로 체포되었다.",[[1843, "charge", "청구하다", 1, "동사"], [1844, "charge", "충전하다", 1, "동사"], [1845, "charge", "책임을 맡기다", 1, "동사"], [1846, "charge", "기소하다, 고소하다", 1, "동사"], [1847, "charge", "(외상으로) 달아놓다", 1, "동사"], [1848, "charge", "비난", 2, "명사"], [1849, "charge", "기소, 혐의", 2, "명사"], [1850, "charge", "요금", 2, "명사"]]],
    [715, 7, "be in charge of", "숙어/관용어", 9, "~을 담당하다", "The coach is in charge of the team practice.", "코치는 팀 연습을 담당하고 있다.",[[1843, "charge", "청구하다", 1, "동사"], [1844, "charge", "충전하다", 1, "동사"], [1845, "charge", "책임을 맡기다", 1, "동사"], [1846, "charge", "기소하다, 고소하다", 1, "동사"], [1847, "charge", "(외상으로) 달아놓다", 1, "동사"], [1848, "charge", "비난", 2, "명사"], [1849, "charge", "기소, 혐의", 2, "명사"], [1850, "charge", "요금", 2, "명사"]]],
    [807, 5, "charge", "명사", 2, "담당, 책임", "He took charge of the farm after his father's death.", "그는 아버지가 죽고 나서 농장을 책임지게 되었다.",[[1843, "charge", "청구하다", 1, "동사"], [1844, "charge", "충전하다", 1, "동사"], [1845, "charge", "책임을 맡기다", 1, "동사"], [1846, "charge", "기소하다, 고소하다", 1, "동사"], [1847, "charge", "(외상으로) 달아놓다", 1, "동사"], [1848, "charge", "비난", 2, "명사"], [1849, "charge", "기소, 혐의", 2, "명사"], [1850, "charge", "요금", 2, "명사"]]],
    [901, 1, "match", "명사", 2, "경기, 시합", "I want to see the final match of the World Cup!", "나는 어서 월드컵 결승전을 보고 싶어!",[[823, "match", "대응하다, 연결하다", 1, "동사"],[824, "match", "맞먹다, 필적하다", 1, "동사"],[825, "match", "부응하다", 1, "동사"],[826, "match", "연결시키다", 1, "동사"],[827, "match", "어울리다", 1, "동사"],[828, "match", "일치하다", 1, "동사"],[829, "match", "짝짓기 하다", 1, "동사"],[830, "match", "맞수, 호적수", 2, "명사"],[831, "match", "성냥, 시합", 2, "명사"],[832, "match", "경기, 시합", 2, "명사"]]],
    [905, 7, "match", "동사", 1, "(동) 어울리다", "The bag doesn't match your jacket.", "그 가방은 너의 재킷과 어울리지 않는다.",[[823, "match", "대응하다, 연결하다", 1, "동사"],[824, "match", "맞먹다, 필적하다", 1, "동사"],[825, "match", "부응하다", 1, "동사"],[826, "match", "연결시키다", 1, "동사"],[827, "match", "어울리다", 1, "동사"],[828, "match", "일치하다", 1, "동사"],[829, "match", "짝짓기 하다", 1, "동사"],[830, "match", "맞수, 호적수", 2, "명사"],[831, "match", "성냥, 시합", 2, "명사"],[832, "match", "경기, 시합", 2, "명사"]]],
    [905, 8, "match up", "동사,숙어/관용어", "#N/A", "맞추다", "It doesn't match up with this hole.", "그것은 이 구멍과 맞지 않는다.",[[823, "match", "대응하다, 연결하다", 1, "동사"],[824, "match", "맞먹다, 필적하다", 1, "동사"],[825, "match", "부응하다", 1, "동사"],[826, "match", "연결시키다", 1, "동사"],[827, "match", "어울리다", 1, "동사"],[828, "match", "일치하다", 1, "동사"],[829, "match", "짝짓기 하다", 1, "동사"],[830, "match", "맞수, 호적수", 2, "명사"],[831, "match", "성냥, 시합", 2, "명사"],[832, "match", "경기, 시합", 2, "명사"]]],
    [701, 6, "match", "동사", 1, "조화시키다, 맞추다, 어울리다", "My dog can match these pictures to their names.", "우리 집 개는 이 사진과 그 이름을 연결시켜 맞출 수 있다.",[[823, "match", "대응하다, 연결하다", 1, "동사"],[824, "match", "맞먹다, 필적하다", 1, "동사"],[825, "match", "부응하다", 1, "동사"],[826, "match", "연결시키다", 1, "동사"],[827, "match", "어울리다", 1, "동사"],[828, "match", "일치하다", 1, "동사"],[829, "match", "짝짓기 하다", 1, "동사"],[830, "match", "맞수, 호적수", 2, "명사"],[831, "match", "성냥, 시합", 2, "명사"],[832, "match", "경기, 시합", 2, "명사"]]],
    [701, 8, "match", "동사", 1, "조화시키다, 맞추다, 어울리다", "My dog can match these pictures to their names.", "우리 집 개는 이 사진과 그 이름을 연결시켜 맞출 수 있다.",[[823, "match", "대응하다, 연결하다", 1, "동사"],[824, "match", "맞먹다, 필적하다", 1, "동사"],[825, "match", "부응하다", 1, "동사"],[826, "match", "연결시키다", 1, "동사"],[827, "match", "어울리다", 1, "동사"],[828, "match", "일치하다", 1, "동사"],[829, "match", "짝짓기 하다", 1, "동사"],[830, "match", "맞수, 호적수", 2, "명사"],[831, "match", "성냥, 시합", 2, "명사"],[832, "match", "경기, 시합", 2, "명사"]]],
    [702, 1, "match", "동사", 1, "연결시키다; 어울리다", "Match the faces with the names.", "이름과 얼굴 표정을 연결하세요.",[[823, "match", "대응하다, 연결하다", 1, "동사"],[824, "match", "맞먹다, 필적하다", 1, "동사"],[825, "match", "부응하다", 1, "동사"],[826, "match", "연결시키다", 1, "동사"],[827, "match", "어울리다", 1, "동사"],[828, "match", "일치하다", 1, "동사"],[829, "match", "짝짓기 하다", 1, "동사"],[830, "match", "맞수, 호적수", 2, "명사"],[831, "match", "성냥, 시합", 2, "명사"],[832, "match", "경기, 시합", 2, "명사"]]],
    [703, 8, "match", "명사", 2, "경기, 시합", "There's a volleyball match every week at the stadium.", "매주 경기장에서 배구 경기가 있다.",[[823, "match", "대응하다, 연결하다", 1, "동사"],[824, "match", "맞먹다, 필적하다", 1, "동사"],[825, "match", "부응하다", 1, "동사"],[826, "match", "연결시키다", 1, "동사"],[827, "match", "어울리다", 1, "동사"],[828, "match", "일치하다", 1, "동사"],[829, "match", "짝짓기 하다", 1, "동사"],[830, "match", "맞수, 호적수", 2, "명사"],[831, "match", "성냥, 시합", 2, "명사"],[832, "match", "경기, 시합", 2, "명사"]]],
    [705, 8, "match", "명사", 2, "시합", "We played a match yesterday.", "우리는 어제 시합을 했다.",[[823, "match", "대응하다, 연결하다", 1, "동사"],[824, "match", "맞먹다, 필적하다", 1, "동사"],[825, "match", "부응하다", 1, "동사"],[826, "match", "연결시키다", 1, "동사"],[827, "match", "어울리다", 1, "동사"],[828, "match", "일치하다", 1, "동사"],[829, "match", "짝짓기 하다", 1, "동사"],[830, "match", "맞수, 호적수", 2, "명사"],[831, "match", "성냥, 시합", 2, "명사"],[832, "match", "경기, 시합", 2, "명사"]]]
]

columns = ["textbook ID", "레슨", "단어", "품사", "PoS", "뜻", "예문", "예문 해석",'voca_options']
df = pd.DataFrame(data, columns=columns)
df.to_csv('example.csv',encoding='utf-8-sig')

In [ ]:
df.head()

,textbook ID,레슨,단어,품사,PoS,뜻,예문,예문 해석,voca_options
0,808,8,charge,동사,1,"(지불, 대금 등을) 청구하다",They charge you $20 to get in the museum.,그들은 박물관에 들어가는 데 너에게 20달러를 청구한다.,"[[1843, charge, 청구하다, 1, 동사], [1844, charge, 충..."
1,903,6,charge,동사,1,(요금을) 청구하다,They charge you $5 to get in the museum.,그들은 박물관에 들어가는 데 5달러를 당신에게 부과할 것이다.,"[[1843, charge, 청구하다, 1, 동사], [1844, charge, 충..."
2,906,5,charge,명사,2,"기소, 혐의",He was arrested on the charge of robbery.,그는 강도 혐의로 체포되었다.,"[[1843, charge, 청구하다, 1, 동사], [1844, charge, 충..."
3,715,7,be in charge of,숙어/관용어,9,~을 담당하다,The coach is in charge of the team practice.,코치는 팀 연습을 담당하고 있다.,"[[1843, charge, 청구하다, 1, 동사], [1844, charge, 충..."
4,807,5,charge,명사,2,"담당, 책임",He took charge of the farm after his father's ...,그는 아버지가 죽고 나서 농장을 책임지게 되었다.,"[[1843, charge, 청구하다, 1, 동사], [1844, charge, 충..."


# for문 돌려서 jsonl 파일 만들기

In [ ]:
def VocaTag(data:pd.DataFrame, output_filename : str) -> str:
    jsonl_data = []

    for i in range(len(data)):
        prompt = prompt_template.render(
            textbook_id = data.loc[i,"textbook ID"],
            lesson = data.loc[i,"레슨"],
            word = data.loc[i,"단어"],
            part_of_speech_of = data.loc[i,"품사"],
            meaning = data.loc[i,"뜻"],
            example_sentence = data.loc[i,"예문"],
            example_sentence_kor = data.loc[i,"예문 해석"],
            voca_options = data.loc[i,"voca_options"]
        )
        voca_options = data.loc[i,"voca_options"]
        voca_id_list = [x[0] for x in voca_options]
        response_format = get_enum_response_format(voca_id_list)
        structure_request = {
            "custom_id" : f"request-{i+1}",
            "method" : "POST",
            "url" : "/v1/chat/completions",
            "body" : {
                "model" : "o3-mini-2025-01-31",
                "reasoning_effort" : "low",
                "messages" : [
                    {"role" : "system", "content" : "Choose the most adequate word"},
                    {"role": "user", "content": prompt}
                ],
                "response_format" : response_format
            }
        }
        jsonl_data.append(structure_request)

    with open(output_filename, 'w', encoding='utf-8') as jsonl_file:
        for item in jsonl_data:
            jsonl_file.write(json.dumps(item, ensure_ascii=False) + '\n')

    print(f'JSONL 파일 생성 완료 : {output_filename}-{i+1}')

In [ ]:
VocaTag(df, output_filename='sample_voca.jsonl')

JSONL 파일 생성 완료 : sample_voca.jsonl-13
